In [ ]:
!pip install datetime
!pip install pandas
!pip install numpy
!pip install scipy
!pip install vaderSentiment
!pip install statsmodels

In [25]:
import pandas as pd
import numpy as np
import datetime
import requests
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
pip install pyldavis

In [ ]:
!pip install pyLDAvis.gensim

In [ ]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import *
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('indextweetsdata.csv')
df = df.drop(columns=['Unnamed: 0','Tweet Id', 'Username'])
df.rename(columns = {'Datetime':'date','Text':'title'}, inplace = True)
df['date'] = pd.to_datetime(df['date']).dt.date
df

In [ ]:
df.sort_values(by = 'date', ascending = True, inplace = True)
data_text=df.groupby(['date'])['title'].apply(list).reset_index(name='LDA_data')
data_text['index'] = data_text.index
document = data_text

document

In [31]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['mint', 'india','today','economic','times','indiatimes.com','bloomberg','reuters','moneycontrol','buisiness','nestle','titan','news','com','new','maruti','suzuki',
                   'tech','mahindra','business','standard','financial','express','business','insider','ntpc','mahindra','airtel','cnbctv','analytics','insight','sbi','indian','may','reddy','dr','drreddy','say','says','telangana','grid','wipro','vs','pv','magazine','ht','tata','tcs','could','pharma','get','titans'])

In [32]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [33]:
def strip_newline(series):
    return [review.replace('\n','') for review in series]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [34]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod


In [35]:
def get_corpus(df):
    df = strip_newline(df)
    words = list(sent_to_words(df))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[sent] for sent in words]

    id2word = gensim.corpora.Dictionary(bigram)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

In [ ]:
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
lda_train4=[]
train_corpus4=[]
for documents in document['LDA_data']:
  train_corpus1, train_id2word4, bigram_train4 = get_corpus(documents)
  train_corpus4.append(train_corpus1)
  with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train4.append(gensim.models.ldamodel.LdaModel(corpus=train_corpus1,num_topics=15,id2word=train_id2word4,passes=5,per_word_topics=True,eval_every = 1))


In [41]:
train_vecs = []
for i in range(len(document['LDA_data'])):
  temp=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
  for j in range(len(train_corpus4[i])):
    top_topics = lda_train4[i].get_document_topics(train_corpus4[i][j])
    for k in range(len(top_topics)):
      temp[k]=temp[k]+top_topics[k][1]
    
  for k in range(15):
    temp[k]=temp[k]/len(train_corpus4[i])
  train_vecs.append(temp)

In [ ]:
print(len(train_vecs[100])) 

In [ ]:
pip install yfinance

In [ ]:
import yfinance as yf
Sensex = yf.download('^BSESN', start= '2020-12-22', end= '2022-12-9')
Sensex.reset_index(inplace=True)
Sensex['Daily Return'] = (Sensex['Adj Close']/ Sensex['Adj Close'].shift(1)) -1

Sensex['DV3'] = ''
for i in range(3):
    col_name = 'shift'+str(i)
    Sensex[col_name] = Sensex['Close'].shift(i)

for i in range(Sensex.shape[0]):
    Sensex['DV3'][i] = np.std([Sensex[('shift'+str(j))][i] for j in range(3)])
    
Sensex['DV3-1'] = Sensex['DV3'].shift(-1)

Sensex.dropna

Sensex['Boolean'] = ''
for i in range(2,(Sensex.shape[0]-1)):

    if Sensex['DV3'][i]>Sensex['DV3-1'][i]:
        Sensex['Boolean'][i]=1
    elif Sensex['DV3'][i]<=Sensex['DV3-1'][i]:
         Sensex['Boolean'][i]=0

Sensex.head(10)

In [46]:
Sensex=Sensex.drop(columns=['Open','High','Low','Close','Adj Close','Volume','Daily Return','DV3','shift0','shift1','shift2','DV3-1'])

In [ ]:
Sensex.reset_index()

In [48]:
Sensex.columns = ['date','UP_DOWN']
Sensex['date'] = pd.to_datetime(Sensex['date']).dt.date

In [ ]:
merged_df = pd.merge(Sensex, document, on='date', how='inner')
merged_df

In [50]:
train=[]
for i in merged_df['index']:
    train.append(train_vecs)

In [51]:
X=np.array(train)
y=np.array(merged_df['UP_DOWN'])

In [ ]:
kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []
cv_lr_a, cv_lrsgd_a, cv_svcsgd_a,  = [], [], []
cv_lr_p, cv_lrsgd_p, cv_svcsgd_p,  = [], [], []
cv_lr_r, cv_lrsgd_r, cv_svcsgd_r,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]

    nsamples, nx, ny = X_train.shape
    X_train = X_train.reshape((nsamples,nx*ny))

    nsamples, nx, ny = X_val.shape
    X_val = X_val.reshape((nsamples,nx*ny))

    y_train=y_train.astype('int')
    y_val=y_val.astype('int')
    
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    lr = LogisticRegression(class_weight= 'balanced',solver='newton-cg',fit_intercept=True).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    cv_lr_p.append(precision_score(y_val, y_pred, average='binary'))
    cv_lr_a.append(accuracy_score(y_val, y_pred))
    cv_lr_r.append(recall_score(y_val, y_pred, average='binary'))

    sgd = linear_model.SGDClassifier(max_iter=1000,tol=1e-3,loss='log',class_weight='balanced').fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    cv_lrsgd_p.append(precision_score(y_val, y_pred, average='binary'))
    cv_lrsgd_a.append(accuracy_score(y_val, y_pred))
    cv_lrsgd_r.append(recall_score(y_val, y_pred, average='binary'))
    
    sgd_huber = linear_model.SGDClassifier(max_iter=1000,tol=1e-3,alpha=20,loss='modified_huber',class_weight='balanced').fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    cv_svcsgd_p.append(precision_score(y_val, y_pred, average='binary'))
    cv_svcsgd_a.append(accuracy_score(y_val, y_pred))
    cv_svcsgd_r.append(recall_score(y_val, y_pred, average='binary'))

print('\n')
print(f'Logistic Regression Val f1:            {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logistic Regression Val Accuracy:      {np.mean(cv_lr_a):.3f} +- {np.std(cv_lr_a):.3f}')
print(f'Logistic Regression Val Recall:        {np.mean(cv_lr_r):.3f} +- {np.std(cv_lr_r):.3f}')
print(f'Logistic Regression Val Precision:     {np.mean(cv_lr_p):.3f} +- {np.std(cv_lr_p):.3f}')
print('\n')
print(f'Logistic Regression SGD Val f1:        {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'Logistic Regression SGD Val Accuracy:  {np.mean(cv_lrsgd_a):.3f} +- {np.std(cv_lrsgd_a):.3f}')
print(f'Logistic Regression SGD Val Recall:    {np.mean(cv_lrsgd_r):.3f} +- {np.std(cv_lrsgd_r):.3f}')
print(f'Logistic Regression SGD Val Precision: {np.mean(cv_lrsgd_p):.3f} +- {np.std(cv_lrsgd_p):.3f}')
print('\n')
print(f'SVM Huber Val f1:                      {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')
print(f'SVM Huber Val Accuracy:                {np.mean(cv_svcsgd_a):.3f} +- {np.std(cv_svcsgd_a):.3f}')
print(f'SVM Huber Val Recall:                  {np.mean(cv_svcsgd_r):.3f} +- {np.std(cv_svcsgd_r):.3f}')
print(f'SVM Huber Val Precision:               {np.mean(cv_svcsgd_p):.3f} +- {np.std(cv_svcsgd_p):.3f}')
print('\n')
